In [1]:
import pandas as pd

In [10]:
# df_reviews = pd.read_json("yelp_dataset/yelp_academic_dataset_review.json", lines=True, nrows=100)
# df_business = pd.read_json("yelp_dataset/yelp_academic_dataset_business.json", lines=True, nrows=10000)

In [40]:
def read_json_business(json_file_path, chunk_size=1000):
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()
    
    # Iterate over chunks
    for chunk_df in pd.read_json(json_file_path, lines=True, chunksize=chunk_size, encoding='utf-8'):
        # Filter by city names (replace 'your_city' with the actual city name)
        filtered_chunk = chunk_df[chunk_df['city'] == 'Philadelphia']
    
        # Append the filtered chunk to the result DataFrame
        result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)
    
    return result_df
    

In [41]:
# df_reviews = pd.read_json("yelp_dataset/yelp_academic_dataset_review.json", lines=True, nrows=1000000)
# df_business = read_json_business("yelp_dataset/yelp_academic_dataset_business.json")

In [42]:
df_business.city.value_counts()

Philadelphia    14569
Name: city, dtype: int64

In [29]:
# df_business.to_json('yelp_dataset/yelp_filtered_philadelphia_business.json', orient='records')

In [43]:
filtered_business = df_business[df_business['categories'].str.contains('Restaurants|Food', na=False)]

In [44]:
def read_json_reviews(json_file_path, ids_business, chunk_size=1000):
    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()

    # Iterate over chunks
    for chunk_df in pd.read_json(json_file_path, lines=True, chunksize=chunk_size, encoding='utf-8'):
        # Filter by business IDs in the list
        filtered_chunk = chunk_df[chunk_df['business_id'].isin(ids_business)]

        # Append the filtered chunk to the result DataFrame
        result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)
        
    return result_df

In [45]:
# ids_business = filtered_business['business_id'].tolist()
# filtered_reviews = read_json_reviews("yelp_dataset/yelp_academic_dataset_review.json", ids_business)

In [46]:
# filtered_reviews.to_json('yelp_dataset/yelp_filtered_philadelphia_reviews.json', orient='records')

In [47]:
filtered_reviews["stars"].value_counts()

5    300621
4    207383
3     97470
1     72020
2     61194
Name: stars, dtype: int64